# Regression Project Walmart Sales Prediction
- This dataset contains weekly sales from 99 departments belonging to 45 different stores. 
- Our aim is to forecast weekly sales from a particular department.
- The objective of this case study is to forecast **weekly retail store sales** based on historical data.
- The data contains holidays and promotional markdowns offered by various stores and several departments throughout the year.
- Markdowns are crucial to promote sales especially before key events such as Super Bowl, Christmas and Thanksgiving. 
- Developing accurate model will enable make informed decisions and make recommendations to improve business processes in the future. 
- The data consists of three sheets: 
    - Stores
    - Features
    - Sales
- Data Source : https://www.kaggle.com/manjeetsingh/retaildataset

**The Task**
- Predict the department-wide sales for each store for the following year
- Model the effects of markdowns on holiday weeks
- Provide recommended actions based on the insights drawn, with prioritization placed on largest business impact

In [1]:
!pip install torch


In [25]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pandas as pd
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

## 1, EDA

In [26]:
# loading data
stores_df = pd.read_csv("stores data-set.csv")
features_df = pd.read_csv("Features data set.csv")
sales_df = pd.read_csv("sales data-set.csv")

### 1.1 store
    Anonymized information about the 45 stores, indicating the type and size of store

In [27]:
stores_df.head(3)

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392


In [28]:
stores_df.describe()

,Store,Size
count,45.000000,45.000000
mean,23.000000,130287.600000
std,13.133926,63825.271991
min,1.000000,34875.000000
25%,12.000000,70713.000000
50%,23.000000,126512.000000
75%,34.000000,202307.000000
max,45.000000,219622.000000


### 1.2 features
    Contains additional data related to the store, department, and regional activity for the given dates.

- Store - the store number
- Date - the week
- Temperature - average temperature in the region
- Fuel_Price - cost of fuel in the region
- MarkDown1-5 - anonymized data related to promotional markdowns. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA
- CPI - the consumer price index
- Unemployment - the unemployment rate
- IsHoliday - whether the week is a special holiday week

In [29]:
features_df["Date"] = pd.to_datetime(features_df["Date"], format="%d/%m/%Y").apply(lambda x: x.date())
features_df.head(3)

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False


In [30]:
features_df.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


**potential problem**
- Nan values for MarkDown, CPI, and Unemployment.
- inconsistent range -> need standardization before most of the regression methods.

### 1.3 sales
    Historical sales data, which covers to 2010-02-05 to 2012-11-01. Within this tab you will find the following fields:

- Store - the store number
- Dept - the department number
- Date - the week
- Weekly_Sales -  sales for the given department in the given store
- IsHoliday - whether the week is a special holiday week

In [31]:
sales_df["Date"] = pd.to_datetime(sales_df["Date"], format="%d/%m/%Y").apply(lambda x: x.date())
sales_df.head(3)

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False


In [32]:
sales_df.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


**potential problem**
- negative value in Sales
- need standardization for sales

### merge data

In [33]:
temp_df = sales_df.merge(stores_df, on="Store", suffixes=(None,"_store"))
df = temp_df.merge(features_df, on=("Store", "Date"), suffixes=(None,"_store"))
df = df.drop(columns=["IsHoliday_store"])

df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2,2010-02-05,50605.27,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2,1,3,2010-02-05,13740.12,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
3,1,4,2010-02-05,39954.04,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
4,1,5,2010-02-05,32229.38,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106


In [34]:
# double check the dimesion
df.shape

(421570, 16)

## 2, feature engineering

In [35]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2,2010-02-05,50605.27,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2,1,3,2010-02-05,13740.12,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
3,1,4,2010-02-05,39954.04,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
4,1,5,2010-02-05,32229.38,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106


Todo:
- decompose Date
- Is Holiday -> binary numeric number
- Type -> one-hot encoding
- Nan -> 0
- Normailzed before regression

In [36]:
# decompose Date
from sklearn.preprocessing import LabelEncoder

df["Year"] = df["Date"].apply(lambda x: x.year)
df["Month"] = df["Date"].apply(lambda x: x.month)
df["Day"] = df["Date"].apply(lambda x: x.day)
# Mark: drop Date

# Is Holiday
df["IsHoliday"] = df["IsHoliday"].astype(int)

# Type -> one-hot encoding
le = LabelEncoder()
df["Type"] = le.fit_transform(df["Type"])
# Mark: drop Type

# fillna
df = df.fillna(0)

# delete columns
# Drop unnecessary columns
df = df.drop(columns=["Date"])

In [37]:
# double check
df.head()

,Store,Dept,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Day
0,1,1,24924.50,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,2010,2,5
1,1,2,50605.27,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,2010,2,5
2,1,3,13740.12,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,2010,2,5
3,1,4,39954.04,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,2010,2,5
4,1,5,32229.38,0,0,151315,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,2010,2,5


In [53]:
df.to_csv("DATA.csv")

In [54]:
df=pd.read_csv("DATA.csv")
col=df1.columns.to_list()
col

['Unnamed: 0',
 'Store',
 'Dept',
 'Weekly_Sales',
 'IsHoliday',
 'Type',
 'Size',
 'Temperature',
 'Fuel_Price',
 'MarkDown1',
 'MarkDown2',
 'MarkDown3',
 'MarkDown4',
 'MarkDown5',
 'CPI',
 'Unemployment',
 'Year',
 'Month',
 'Day']

In [57]:
Store=[df['Store'].to_list()]
Dept=[df['Dept'].to_list()]
IsHoliday=[df['IsHoliday'].to_list()]
Type=[df['Type'].to_list()]
Size=[df['Size'].to_list()]
Temperature=[df['Temperature'].to_list()]
Fuel_Price=[df['Fuel_Price'].to_list()]
MarkDown1=[df['MarkDown1'].to_list()]
MarkDown2=[df['MarkDown2'].to_list()]
MarkDown3=[df['MarkDown3'].to_list()]
MarkDown4=[df['MarkDown4'].to_list()]
MarkDown5=[df['MarkDown5'].to_list()]
CPI=[df['CPI'].to_list()]
Unemployment=[df['Unemployment'].to_list()]
Year=[df['Year'].to_list()]
Month=[df['Month'].to_list()]
Day= [df['Day'].to_list()]
Weekly_Sales=[df['Weekly_Sales'].to_list()]


In [60]:
dict={
    
    col[1]:Store,
    col[2]:Dept,
    col[3]:Weekly_Sales,
    col[4]:IsHoliday,
    col[5]:Type,
    col[6]:Size,
    col[7]:Temperature,
    col[8]:Fuel_Price,
    col[9]:MarkDown1,
    col[10]:MarkDown2,
    col[11]:MarkDown3,
    col[12]:MarkDown4,
    col[13]:MarkDown5,
    col[14]:CPI,
    col[15]:Unemployment,
    col[16]:Year,
    col[17]:Month,
    col[18]:Day
}

In [62]:
df1=pd.DataFrame(dict)

In [63]:
df1

,Store,Dept,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Day
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[24924.5, 50605.27, 13740.12, 39954.04, 32229....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[151315, 151315, 151315, 151315, 151315, 15131...","[42.31, 42.31, 42.31, 42.31, 42.31, 42.31, 42....","[2.572, 2.572, 2.572, 2.572, 2.572, 2.572, 2.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[211.0963582, 211.0963582, 211.0963582, 211.09...","[8.106, 8.106, 8.106, 8.106, 8.106, 8.106, 8.1...","[2010, 2010, 2010, 2010, 2010, 2010, 2010, 201...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."


In [64]:
df1.to_excel("DICT_DATA.xlsx")

## 4, preprocessing

In [38]:
X = df[df.columns[~df.columns.isin(["Weekly_Sales"])]]
Y = df[["Weekly_Sales"]]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [39]:

# normalize data
train_scaler = StandardScaler()
X_train = train_scaler.fit_transform(X_train)
X_test = train_scaler.transform(X_test)

### Elastic Net (adding L1 & L2 normalization)

In [40]:
elasticnet_model = ElasticNet(alpha = 0.2, l1_ratio=0.5).fit(X_train, y_train)

# R square for the model
R_square_train = elasticnet_model.score(X_train, y_train)
R_square_test = elasticnet_model.score(X_test, y_test)

print(f"Training R^2: {R_square_train:.4f}\nTesting R^2: {R_square_test:.4f}")

# calculating the MAPE
y_hat = elasticnet_model.predict(X_test)
MSE = mean_squared_error(y_test, y_hat)
MAE = mean_absolute_error(y_test, y_hat)
print(f"{MSE=:.4f}\n{MAE=:.4f}")

Training R^2: 0.0858
Testing R^2: 0.0866
MSE=480196004.3157
MAE=14578.2256


In [29]:
import pickle

# Assuming `elastic_net_model` is your trained Elastic Net model
with open('elastic_net_model.pkl', 'wb') as f:
    pickle.dump(elasticnet_model, f)


We achieve 94% R-square in testing dataset, which means the model has a relativly good prediction on the testing data.

In [41]:
import joblib

# After training your model
joblib.dump(elasticnet_model, 'elastic_net_model1.pkl')
joblib.dump(train_scaler, 'scaler1.pkl')


['scaler1.pkl']

In [ ]:
# Tableau SCRIPT_REAL example
# Assuming you pass the parameters `Store`, `Dept`, `Type`, `Size`, etc., from Tableau:

SCRIPT_REAL("
import joblib
import numpy as np

# Load the model and scaler
model = joblib.load('path/to/elastic_net_model.pkl')
scaler = joblib.load('path/to/scaler.pkl')

def predict_sales(store, dept, type_, size, temperature, fuel_price, cpi, unemployment, year, month, day, is_holiday,
                 markdown1=0, markdown2=0, markdown3=0, markdown4=0, markdown5=0):
    type_dict = {'A': 0, 'B': 1, 'C': 2}
    type_encoded = type_dict[type_]

    input_data = np.array([[store, dept, type_encoded, size, temperature, fuel_price, markdown1, markdown2,
                            markdown3, markdown4, markdown5, cpi, unemployment, year, month, day, is_holiday]])
    input_data_scaled = scaler.transform(input_data)
    return model.predict(input_data_scaled)[0]

result = predict_sales(INT([Store]), INT([Dept]), STR([Type]), FLOAT([Size]), 
                       FLOAT([Temperature]), FLOAT([Fuel_Price]), 
                       FLOAT([CPI]), FLOAT([Unemployment]), 
                       INT([Year]), INT([Month]), INT([Day]), INT([IsHoliday]), 
                       FLOAT([MarkDown1]), FLOAT([MarkDown2]), FLOAT([MarkDown3]), 
                       FLOAT([MarkDown4]), FLOAT([MarkDown5]))

return result
", [Store], [Dept], [Type], [Size], [Temperature], [Fuel_Price], 
    [CPI], [Unemployment], [Year], [Month], [Day], [IsHoliday], 
    [MarkDown1], [MarkDown2], [MarkDown3], [MarkDown4], [MarkDown5])


In [ ]:
SCRIPT_REAL("
return tabpy.query('predict_sales', _arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8, _arg9, _arg10, _arg11, _arg12, _arg13, _arg14, _arg15).tolist()
",dd
[Store], [Dept], [Type], [Size], [Temperature], [Fuel_Price], [MarkDown1], [MarkDown2], 
[MarkDown3], [MarkDown4], [MarkDown5], [CPI], [Unemployment], [Year], [Month], [Day], [IsHoliday])


In [ ]:
SCRIPT_REAL("
import joblib
import numpy as np

# Load the model and scaler
model = joblib.load('C:\\\\Users\\\\hamsa\\\\Downloads\\\\Walmart_Proj\\\\elastic_net_model1.pkl')
scaler = joblib.load('C:\\\\Users\\\\hamsa\\\\Downloads\\\\Walmart_Proj\\\\scaler1.pkl')

# Define the prediction function
def predict_sales():
    input_list = [_arg[0][0], _arg[1][0], _arg[2][0], _arg[3][0], _arg[4][0], _arg[5][0], _arg[6][0], _arg[7][0],
                  _arg[8][0], _arg[9][0], _arg[10][0], _arg[11][0], _arg[12][0], _arg[13][0], _arg[14][0], _arg[15][0], _arg[16][0]]
    inp = np.array(input_list).reshape(1, -1)
    inp = scaler.transform(inp)
    return model.predict(inp)

# Call the prediction function
result = predict_sales()
return float(result[0])
", 
ATTR([Store]), ATTR([Dept]), ATTR([Type]), ATTR([Size]), ATTR([Temperature]), ATTR([Fuel Price]), ATTR([MarkDown1]), ATTR([MarkDown2]), 
ATTR([MarkDown3]), ATTR([MarkDown4]), ATTR([MarkDown5]), ATTR([CPI]), ATTR([Unemployment]), ATTR([Year]), ATTR([Month]), ATTR([Day]), 
ATTR([Is Holiday]), [Parameters].[CPI], [Parameters].[Day], [Parameters].[Dept], [Parameters].[Fuel Price], [Parameters].[Is Holiday], 
[Parameters].[MarkDown1], [Parameters].[MarkDown2], [Parameters].[MarkDown3], [Parameters].[MarkDown4], [Parameters].[MarkDown5], 
[Parameters].[Month], [Parameters].[Size], [Parameters].[Store], [Parameters].[Temperature], [Parameters].[Type], 
[Parameters].[Unemployment], [Parameters].[Year])